# Satellite Container Analysis for Trading Signals

This notebook implements the satellite image analysis system described in the research to:
1. Collect satellite imagery from major ports
2. Use deep learning to count shipping containers
3. Generate trading signals from container volume changes

In [31]:
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import ee
import torch
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
import requests
from PIL import Image
from io import BytesIO
import pynvml

# Import GEE collector
from gee_collector import GoogleEarthCollector

In [ ]:
# Train YOLO on xView dataset (for satellite imagery detection)
# xView has 60 classes including Container Ship, Shipping Container, Maritime Vessel, etc.
from ultralytics import YOLO

model = YOLO("yolo11n.pt")  # load a pretrained model
results = model.train(data="xView.yaml", epochs=100, imgsz=640)

# Note: xView dataset must be manually downloaded from https://challenge.xviewdataset.org
# Extract to datasets/xView/ directory before training

New https://pypi.org/project/ultralytics/8.4.11 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.4.7 🚀 Python-3.10.19 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 3060, 11908MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dota8.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo26n-obb.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train, nbs=64, nms=False, op

/home/misango/anaconda3/envs/research_env/lib/python3.10/site-packages/mlflow/tracking/_tracking_service/utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)
2026/02/04 22:25:01 INFO mlflow.tracking.fluent: Experiment with name '/Shared/Ultralytics' does not exist. Creating a new experiment.


MLflow: logging run_id(64553a3376724deab76f25f2b9fc0f64) to runs/mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri runs/mlflow'
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /home/misango/codechest/Algorithmic_Trading_and_HFT_Research/Satellite_Container_Analysis/runs/obb/train
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss angle_loss  Instances       Size
      1/100     0.721G     0.9284     0.7868    0.01875   0.004521         72        640: 100% ━━━━━━━━━━━━ 1/1 9.3s/it 9.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 2.4s/it 2.4s
                   all          4          8      0.977      0.973      0.995      0.877

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss angle_loss  Instances       Size
      2/100     0.797G      1.073     0.5874    0.

In [13]:
# Load configuration
with open('config.json', 'r') as f:
    config = json.load(f)

# Initialize Google Earth Engine Collector
collector = GoogleEarthCollector(config_path='config.json', authenticate=False)

ports = config['ports']
print(f"Loaded {len(ports)} major ports for analysis")

✓ Google Earth Engine initialized with project: algorithmictrading-414808
Loaded 5 major ports for analysis


## Port Locations

Let's visualize the major ports we're analyzing:

In [34]:
# Display port information
import plotly.graph_objects as go

print("Major Ports for Container Analysis:")
print("=" * 70)
for i, port in enumerate(ports, 1):
    print(f"{i}. {port['name']:15} | {port['country']:15} | Lat: {port['lat']:7.4f}, Lon: {port['lon']:8.4f}")
print("=" * 70)

# Create world map with port locations
fig = go.Figure(data=go.Scattergeo(
    lon=[p['lon'] for p in ports],
    lat=[p['lat'] for p in ports],
    text=[f"{p['name']}<br>{p['country']}" for p in ports],
    mode='markers+text',
    marker=dict(
        size=15,
        color='red',
        line=dict(width=2, color='white')
    ),
    textposition="top center",
    textfont=dict(size=12, color='black', family='Arial Black')
))

fig.update_layout(
    title='Major Ports for Container Analysis',
    geo=dict(
        projection_type='natural earth',
        showland=True,
        landcolor='rgb(243, 243, 243)',
        coastlinecolor='rgb(204, 204, 204)',
        showcountries=True,
        countrycolor='rgb(204, 204, 204)',
    ),
    height=500,
)

fig.show()

Major Ports for Container Analysis:
1. Shanghai        | China           | Lat: 31.2304, Lon: 121.4737
2. Singapore       | Singapore       | Lat:  1.2644, Lon: 103.8220
3. Rotterdam       | Netherlands     | Lat: 51.9225, Lon:   4.4792
4. Los Angeles     | USA             | Lat: 33.7405, Lon: -118.2720
5. Hamburg         | Germany         | Lat: 53.5395, Lon:   9.9847


In [ ]:
class ContainerDetector:
    def __init__(self, model_type='xview'):
        """
        Initialize container detector with satellite-specific models
        
        Args:
            model_type: 'xview' (best for satellite), 'dota' (aerial), or 'yolo-coco' (ground-level)
        """
        self.model_type = model_type
        
        if model_type == 'xview':
            # xView dataset - specifically for overhead satellite imagery
            try:
                # Try your trained xView model
                self.model = YOLO('runs/detect/train/weights/best.pt')
                print("✓ Loaded your trained xView model")
                print("  xView classes include: Container Ship, Shipping Container, Maritime Vessel")
            except:
                try:
                    # Try pre-existing xView model
                    self.model = YOLO('yolov8n-xview.pt')
                    print("✓ Loaded YOLOv8 trained on xView dataset")
                except:
                    print("⚠️  xView model not found")
                    print("   Train one with: model.train(data='xView.yaml', epochs=100)")
                    print("   Falling back to OBB model...")
                    self.model = YOLO('runs/obb/train/weights/best.pt')
                    self.model_type = 'dota'
        elif model_type == 'dota':
            # DOTA dataset for aerial imagery
            try:
                self.model = YOLO('runs/obb/train/weights/best.pt')
                print("✓ Loaded your trained DOTA OBB model")
            except:
                self.model = YOLO('yolov8n.pt')
                self.model_type = 'yolo-optimized'
        else:
            # Standard COCO-trained model
            self.model = YOLO('yolov8n.pt')
            print("✓ Loaded standard YOLOv8 (COCO dataset)")
        
        # Print available classes
        print(f"  Model classes ({len(self.model.names)}): {list(self.model.names.values())[:10]}...")
    
    def preprocess_image(self, image_path):
        """Preprocess satellite image for container detection"""
        img = cv2.imread(str(image_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # For satellite imagery, enhance contrast
        if self.model_type in ['xview', 'dota', 'yolo-optimized']:
            # Convert to LAB color space for better contrast
            lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
            l, a, b = cv2.split(lab)
            
            # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
            clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
            l = clahe.apply(l)
            
            # Merge and convert back
            enhanced = cv2.merge([l, a, b])
            img = cv2.cvtColor(enhanced, cv2.COLOR_LAB2RGB)
        
        return img
    
    def detect_containers(self, image, show_all_classes=False):
        """Detect and count containers in satellite image"""
        
        # Define container-related classes for xView dataset
        xview_container_classes = [
            'Container Ship',       # Class 31
            'Shipping Container',   # Class 57
            'Shipping container lot', # Class 56
            'Container Crane',      # Class 35
            'Cargo Ship',          # Class 2
            'Cargo Plane',         # Class 2
            'Cargo Truck',         # Class 10
            'Cargo Car',           # Class 19
            'Maritime Vessel',     # Class 23
            'Barge',               # Class 27
            'Crane Truck',         # Class 16
            'Reach Stacker',       # Class 36
            'Straddle Carrier'     # Class 37
        ]
        
        if self.model_type == 'xview':
            conf_threshold = 0.15  # Reasonable threshold for xView
            container_classes = xview_container_classes
        elif self.model_type == 'dota':
            conf_threshold = 0.10
            container_classes = list(self.model.names.values())
        else:
            conf_threshold = 0.05
            container_classes = [2, 5, 7, 8]  # car, bus, truck, boat for COCO
        
        # Use higher image size for better detection
        results = self.model(image, conf=conf_threshold, verbose=False, imgsz=1280)
        
        containers = []
        all_detections = []
        
        for result in results:
            boxes = result.boxes if hasattr(result, 'boxes') else result.obb
            if boxes is not None:
                for box in boxes:
                    cls = int(box.cls)
                    conf = float(box.conf)
                    class_name = self.model.names[cls]
                    
                    # Track all detections for analysis
                    all_detections.append({
                        'class_id': cls,
                        'class_name': class_name,
                        'confidence': conf
                    })
                    
                    # Filter for container-like objects
                    is_container = False
                    if self.model_type == 'xview':
                        is_container = class_name in container_classes
                    elif self.model_type == 'dota':
                        is_container = True  # Accept all DOTA classes
                    else:
                        is_container = cls in container_classes
                    
                    if is_container and conf > conf_threshold:
                        containers.append({
                            'bbox': box.xyxy.cpu().numpy() if hasattr(box, 'xyxy') else box.xywhr.cpu().numpy(),
                            'confidence': conf,
                            'class': cls,
                            'class_name': class_name
                        })
        
        if show_all_classes:
            return len(containers), containers, all_detections
        return len(containers), containers
    
    def analyze_port_activity(self, image_dir, port_name):
        """Analyze container activity for a specific port"""
        image_files = list(Path(image_dir).glob('*.jp*g')) + list(Path(image_dir).glob('*.png'))
        results = []
        
        for img_path in image_files:
            img = self.preprocess_image(img_path)
            count, detections = self.detect_containers(img)
            
            results.append({
                'port': port_name,
                'image': img_path.name,
                'container_count': count,
                'timestamp': datetime.now()
            })
        
        return pd.DataFrame(results)

In [36]:
class TradingSignalGenerator:
    def __init__(self):
        self.container_data = pd.DataFrame()
        
    def load_container_data(self, data):
        """Load container count data"""
        self.container_data = data
        
    def calculate_signals(self):
        """Generate trading signals from container volume changes"""
        if self.container_data.empty:
            return pd.DataFrame()
        
        # Group by port and calculate rolling statistics
        signals = []
        
        for port in self.container_data['port'].unique():
            port_data = self.container_data[self.container_data['port'] == port].copy()
            port_data = port_data.sort_values('timestamp')
            
            # Calculate moving averages and changes
            port_data['ma_7'] = port_data['container_count'].rolling(7).mean()
            port_data['ma_30'] = port_data['container_count'].rolling(30).mean()
            port_data['pct_change'] = port_data['container_count'].pct_change()
            
            # Generate signals
            port_data['signal'] = 0
            port_data.loc[port_data['ma_7'] > port_data['ma_30'], 'signal'] = 1  # Bullish
            port_data.loc[port_data['ma_7'] < port_data['ma_30'], 'signal'] = -1  # Bearish
            
            # Warning signal for extremely high volumes
            high_threshold = port_data['container_count'].quantile(0.95)
            port_data['warning'] = port_data['container_count'] > high_threshold
            
            signals.append(port_data)
        
        return pd.concat(signals, ignore_index=True)
    
    def generate_global_signal(self, port_signals):
        """Generate global trading signal from all ports"""
        global_signal = port_signals.groupby('timestamp').agg({
            'container_count': 'sum',
            'signal': 'mean',
            'warning': 'any'
        }).reset_index()
        
        global_signal['global_signal'] = np.where(
            global_signal['signal'] > 0.2, 1,
            np.where(global_signal['signal'] < -0.2, -1, 0)
        )
        
        return global_signal

In [37]:
# Initialize detector and signal generator
detector = ContainerDetector(model_type='dota')
signal_gen = TradingSignalGenerator()

print("✓ Container detector initialized")
print("✓ Trading signal generator initialized")

✓ Loaded your trained DOTA OBB model
  Model classes: ['plane', 'ship', 'storage tank', 'baseball diamond', 'tennis court', 'basketball court', 'ground track field', 'harbor', 'bridge', 'large vehicle', 'small vehicle', 'helicopter', 'roundabout', 'soccer ball field', 'swimming pool']
✓ Container detector initialized
✓ Trading signal generator initialized


## Install DOTA-Trained Model

Three options to get satellite-specific detection:

In [38]:
# Download satellite images using GoogleEarthCollector
shanghai = ports[0]  # Shanghai port
print(f"\nDownloading imagery of {shanghai['name']} port...")
print(f"Using Google Earth Engine with Sentinel-2 satellite")

# Download images using GEE collector
downloaded_df = collector.collect_port_images(
    port_name=shanghai['name'],
    days_back=30,
    max_images=10,
    max_cloud=10,
    source='sentinel2'
)

if not downloaded_df.empty:
    print(f"\n✓ Downloaded {len(downloaded_df)} images")
    
    # Convert DataFrame to list format for compatibility with visualization code
    downloaded = [
        {
            'port': row['port'],
            'filepath': Path(row['filepath']),
            'date': row['date']
        }
        for _, row in downloaded_df.iterrows()
    ]
else:
    print("No images downloaded")
    downloaded = []


Using Google Earth Engine with Sentinel-2 satellite

Processing: Shanghai (China)
Coordinates: 31.2304, 121.4737
Date range: 2026-01-05 to 2026-02-04
Max cloud cover: 10%
Found 7 images with <10% cloud cover
  ✓ Saved: Shanghai_2026-02-03_sentinel2.png
  ✓ Saved: Shanghai_2026-02-03_sentinel2.png
  ✓ Saved: Shanghai_2026-01-24_sentinel2.png
  ✓ Saved: Shanghai_2026-01-14_sentinel2.png
  ✓ Saved: Shanghai_2026-01-12_sentinel2.png
  ✓ Saved: Shanghai_2026-01-09_sentinel2.png
  ✓ Saved: Shanghai_2026-01-07_sentinel2.png
✓ Downloaded 7 images for Shanghai

✓ Saved metadata to data/results/image_metadata.csv
✓ Total images downloaded: 7

✓ Downloaded 7 images


## Sample Image Download and Visualization

Now let's download actual satellite images and visualize them:

In [27]:
# Visualize ALL downloaded images
if 'downloaded' in dir() and len(downloaded) > 0:
    print(f"\n{'='*70}")
    print(f"Downloaded {len(downloaded)} satellite images for {shanghai['name']}")
    print(f"{'='*70}\n")
    
    # Calculate grid dimensions for all images
    n_images = len(downloaded)
    n_cols = min(5, n_images)  # Max 5 columns
    n_rows = (n_images + n_cols - 1) // n_cols  # Ceiling division
    
    # Create grid of subplots
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 4, n_rows * 4))
    
    # Flatten axes array for easy iteration
    if n_images == 1:
        axes = [axes]
    else:
        axes = axes.flatten() if n_rows > 1 else axes
    
    # Display all images
    for idx, img_info in enumerate(downloaded):
        img = Image.open(img_info['filepath'])
        axes[idx].imshow(img)
        axes[idx].set_title(f"{img_info['port']}\n{img_info['date']}", 
                           fontsize=10, fontweight='bold')
        axes[idx].axis('off')
    
    # Hide unused subplots
    for idx in range(n_images, len(axes)):
        axes[idx].axis('off')
    
    plt.suptitle(f'Sentinel-2 Satellite Imagery - {shanghai["name"]} Port ({n_images} images)', 
                 fontsize=14, fontweight='bold', y=0.98)
    plt.tight_layout()
    plt.show()
    
    # Print detailed summary
    print(f"\n Image Summary:")
    print(f"  • Total images: {n_images}")
    print(f"  • Date range: {downloaded[0]['date']} to {downloaded[-1]['date']}")
    print(f"\n Image Details:")
    for i, img_info in enumerate(downloaded, 1):
        print(f"  {i:2d}. {img_info['date']} - {img_info['filepath'].name}")
else:
    print(" No images downloaded yet. Run the previous cell first.")


Downloaded 7 satellite images for Shanghai



<Figure size 2000x800 with 10 Axes>


 Image Summary:
  • Total images: 7
  • Date range: 2026-02-03 to 2026-01-07

 Image Details:
   1. 2026-02-03 - Shanghai_2026-02-03_sentinel2.png
   2. 2026-02-03 - Shanghai_2026-02-03_sentinel2.png
   3. 2026-01-24 - Shanghai_2026-01-24_sentinel2.png
   4. 2026-01-14 - Shanghai_2026-01-14_sentinel2.png
   5. 2026-01-12 - Shanghai_2026-01-12_sentinel2.png
   6. 2026-01-09 - Shanghai_2026-01-09_sentinel2.png
   7. 2026-01-07 - Shanghai_2026-01-07_sentinel2.png


In [32]:
# Inference on all downloaded satellite images using trained OBB model
from ultralytics import YOLO
from pathlib import Path
import pandas as pd

# Load the best trained OBB model
model = YOLO("runs/obb/train/weights/best.pt")

# Directory containing satellite images
image_dir = Path("data/images")
# GEE collector saves images as .png files
image_files = list(image_dir.glob("*.png")) + list(image_dir.glob("*.jpg"))

if not image_files:
    print(f"No images found in {image_dir}")
    print("Please run the image download cell first (Cell 10)")
else:
    print(f"Found {len(image_files)} images to process\n")
    
    results_list = []
    
    for img_path in image_files:
        try:
            results = model(str(img_path))
            # Get results as DataFrame
            result_data = results[0].to_df()
            
            # Handle both polars and pandas DataFrames
            if hasattr(result_data, "to_pandas"):
                df = result_data.to_pandas()
            else:
                df = pd.DataFrame(result_data)
            
            df["image"] = img_path.name
            results_list.append(df)
            print(f"✓ Processed {img_path.name} - {len(df)} detections")
        except Exception as e:
            print(f"✗ Error processing {img_path.name}: {e}")
    
    # Combine all results into a single DataFrame
    if results_list:
        all_detections = pd.concat(results_list, ignore_index=True)
        print("\nDetection results for all images:")
        print(all_detections.head())
        
        # Save to CSV
        Path("data/results").mkdir(parents=True, exist_ok=True)
        all_detections.to_csv("data/results/obb_detections.csv", index=False)
        print(f"\n✓ All detection results saved to data/results/obb_detections.csv")
        print(f"✓ Total detections: {len(all_detections)}")
    else:
        print("\n No detections made from any images")

Found 6 images to process


image 1/1 /home/misango/codechest/Algorithmic_Trading_and_HFT_Research/Satellite_Container_Analysis/data/images/Shanghai_2026-01-09_sentinel2.png: 640x576 1 bridge, 1 swimming pool, 13.7ms
Speed: 2.6ms preprocess, 13.7ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 576)
✓ Processed Shanghai_2026-01-09_sentinel2.png - 2 detections

image 1/1 /home/misango/codechest/Algorithmic_Trading_and_HFT_Research/Satellite_Container_Analysis/data/images/Shanghai_2026-01-14_sentinel2.png: 640x576 2 bridges, 1 roundabout, 1 swimming pool, 11.6ms
Speed: 2.4ms preprocess, 11.6ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 576)
✓ Processed Shanghai_2026-01-14_sentinel2.png - 4 detections

image 1/1 /home/misango/codechest/Algorithmic_Trading_and_HFT_Research/Satellite_Container_Analysis/data/images/Shanghai_2026-01-12_sentinel2.png: 640x576 1 harbor, 1 bridge, 1 roundabout, 12.7ms
Speed: 2.2ms preprocess, 12.7ms inference, 0.3ms postprocess per 

## Container Detection on Downloaded Images

Apply YOLO model to detect containers in the satellite images:

In [33]:
# Run container detection on downloaded images
if 'downloaded' in dir() and len(downloaded) > 0:
    print("Running YOLO container detection...")
    print("NOTE: YOLO is trained on ground photos, not satellite imagery!")
    print("=" * 70)
    
    detection_results = []
    all_classes_found = {}
    
    for img_info in downloaded:
        # Load and preprocess image
        img = detector.preprocess_image(img_info['filepath'])
        
        # Detect containers and get all detections
        count, detections, all_dets = detector.detect_containers(img, show_all_classes=True)
        
        # Track what classes were found
        for det in all_dets:
            class_name = det['class_name']
            if class_name not in all_classes_found:
                all_classes_found[class_name] = 0
            all_classes_found[class_name] += 1
        
        detection_results.append({
            'port': img_info['port'],
            'date': img_info['date'],
            'filepath': img_info['filepath'],
            'container_count': count,
            'detections': detections,
            'all_detections': all_dets
        })
        
        print(f"✓ {img_info['date']}: {count:3d} vehicles | {len(all_dets):3d} total objects")
    
    print("=" * 70)
    
    # Show what YOLO actually detected
    print(f"\nAll Object Classes Detected (across all {len(downloaded)} images):")
    if all_classes_found:
        for class_name, count in sorted(all_classes_found.items(), key=lambda x: x[1], reverse=True):
            print(f"  • {class_name:20s}: {count:4d} instances")
    else:
        print("  No objects detected at all!")
        print("  This is because:")
        print("    - Satellite images are taken from 700km altitude")
        print("    - YOLO expects ground-level photos (0-100m)")
        print("    - Objects are too small (< 5 pixels)")
    
    # Visualize detections on image with most detections
    if detection_results:
        # Find image with most detections
        best_result = max(detection_results, key=lambda x: len(x['all_detections']))
        
        img = cv2.imread(str(best_result['filepath']))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Draw ALL detections (not just vehicles)
        for det in best_result['all_detections'][:50]:  # Limit to 50 for clarity
            # Find bounding boxes from the model results
            pass
        
        # Draw vehicle detections with boxes
        for det in best_result['detections']:
            bbox = det['bbox'][0]
            x1, y1, x2, y2 = map(int, bbox)
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 3)
            label = f"{det['class_name']} {det['confidence']:.2f}"
            cv2.putText(img, label, (x1, y1-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
        
        # Display
        plt.figure(figsize=(14, 10))
        plt.imshow(img)
        plt.title(f"Best Detection Result - {best_result['port']} ({best_result['date']})\n{best_result['container_count']} vehicles | {len(best_result['all_detections'])} total objects", 
                 fontsize=14, fontweight='bold')
        plt.axis('off')
        plt.tight_layout()
        plt.show()
        
        total_vehicles = sum(r['container_count'] for r in detection_results)
        total_objects = sum(len(r['all_detections']) for r in detection_results)
        print(f"\nDetection Summary:")
        print(f"  • Total vehicles detected: {total_vehicles} (across {len(detection_results)} images)")
        print(f"  • Total objects detected: {total_objects}")
        print(f"  • Average per image: {total_vehicles/len(detection_results):.1f} vehicles, {total_objects/len(detection_results):.1f} objects")
        
        if total_vehicles == 0:
            print(f"\nRecommendation:")
            print(f"  For satellite imagery, you need:")
            print(f"  1. Models trained on aerial/satellite data (not COCO)")
            print(f"  2. Higher resolution images (4096x4096 or larger)")
            print(f"  3. Specialized container detection models")
            print(f"  4. Or use synthetic data for demonstration (next cell)")
else:
    print("⚠️  No images available for detection. Download images first.")

Running YOLO container detection...
NOTE: YOLO is trained on ground photos, not satellite imagery!
✓ 2026-02-03:   0 vehicles |   0 total objects
✓ 2026-02-03:   0 vehicles |   0 total objects
✓ 2026-01-24:   0 vehicles |   0 total objects
✓ 2026-01-14:   0 vehicles |   0 total objects
✓ 2026-01-12:   0 vehicles |   0 total objects
✓ 2026-01-09:   0 vehicles |   0 total objects
✓ 2026-01-07:   0 vehicles |   0 total objects

All Object Classes Detected (across all 7 images):
  No objects detected at all!
  This is because:
    - Satellite images are taken from 700km altitude
    - YOLO expects ground-level photos (0-100m)
    - Objects are too small (< 5 pixels)


<Figure size 1400x1000 with 1 Axes>


Detection Summary:
  • Total vehicles detected: 0 (across 7 images)
  • Total objects detected: 0
  • Average per image: 0.0 vehicles, 0.0 objects

Recommendation:
  For satellite imagery, you need:
  1. Models trained on aerial/satellite data (not COCO)
  2. Higher resolution images (4096x4096 or larger)
  3. Specialized container detection models
  4. Or use synthetic data for demonstration (next cell)


## Trading Signal Generation

Generate trading signals from container volume analysis:

In [ ]:
# Generate trading signals
signal_gen.load_container_data(df_containers)
port_signals = signal_gen.calculate_signals()
global_signals = signal_gen.generate_global_signal(port_signals)

print("Trading signals generated:")
print(global_signals.tail())

In [ ]:
# Visualization
print("\nGenerating trading signal visualizations...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Container counts by port
print("  • Plotting container counts by port...")
for port in ports:
    port_data = df_containers[df_containers['port'] == port['name']]
    axes[0,0].plot(port_data['timestamp'], port_data['container_count'], 
                   label=f"{port['name']} ({port['country']})", marker='o', markersize=3)
axes[0,0].set_title('Container Counts by Port', fontsize=14, fontweight='bold')
axes[0,0].set_xlabel('Date', fontsize=11)
axes[0,0].set_ylabel('Container Count', fontsize=11)
axes[0,0].legend(loc='best', fontsize=9)
axes[0,0].grid(True, alpha=0.3)
axes[0,0].tick_params(axis='x', rotation=45)

# Global container volume
print("  • Plotting global container volume...")
axes[0,1].plot(global_signals['timestamp'], global_signals['container_count'], 
               color='blue', linewidth=2, marker='o', markersize=4)
axes[0,1].set_title('Global Container Volume (All Ports)', fontsize=14, fontweight='bold')
axes[0,1].set_xlabel('Date', fontsize=11)
axes[0,1].set_ylabel('Total Container Count', fontsize=11)
axes[0,1].grid(True, alpha=0.3)
axes[0,1].tick_params(axis='x', rotation=45)
axes[0,1].fill_between(global_signals['timestamp'], global_signals['container_count'], 
                        alpha=0.3, color='blue')

# Trading signals
print("  • Plotting trading signals...")
signal_colors = {1: 'green', -1: 'red', 0: 'gray'}
for signal_val, color in signal_colors.items():
    mask = global_signals['global_signal'] == signal_val
    axes[1,0].scatter(global_signals[mask]['timestamp'], 
                     global_signals[mask]['global_signal'],
                     c=color, s=100, alpha=0.7, 
                     label=f"{'Buy' if signal_val == 1 else 'Sell' if signal_val == -1 else 'Hold'}")
axes[1,0].axhline(y=0, color='black', linestyle='--', linewidth=1)
axes[1,0].set_title('Global Trading Signal', fontsize=14, fontweight='bold')
axes[1,0].set_xlabel('Date', fontsize=11)
axes[1,0].set_ylabel('Signal (-1: Sell, 0: Hold, 1: Buy)', fontsize=11)
axes[1,0].set_ylim(-1.5, 1.5)
axes[1,0].grid(True, alpha=0.3)
axes[1,0].legend(loc='best')
axes[1,0].tick_params(axis='x', rotation=45)

# Warning signals
print("  • Plotting warning signals...")
warning_dates = global_signals[global_signals['warning']]['timestamp']
normal_dates = global_signals[~global_signals['warning']]['timestamp']
axes[1,1].scatter(warning_dates, [1]*len(warning_dates), 
                 color='red', s=150, alpha=0.8, marker='X', label='High Volume Warning')
axes[1,1].scatter(normal_dates, [0]*len(normal_dates), 
                 color='green', s=50, alpha=0.5, marker='o', label='Normal Volume')
axes[1,1].set_title('High Volume Warning Signals', fontsize=14, fontweight='bold')
axes[1,1].set_xlabel('Date', fontsize=11)
axes[1,1].set_ylabel('Status', fontsize=11)
axes[1,1].set_ylim(-0.5, 1.5)
axes[1,1].set_yticks([0, 1])
axes[1,1].set_yticklabels(['Normal', 'Warning'])
axes[1,1].grid(True, alpha=0.3)
axes[1,1].legend(loc='best')
axes[1,1].tick_params(axis='x', rotation=45)

plt.suptitle('Satellite Container Analysis - Trading Dashboard', 
             fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("\n✓ Visualizations complete!")

In [ ]:
# Save results
output_dir = Path('data/results')
output_dir.mkdir(parents=True, exist_ok=True)

# Save container data and signals
df_containers.to_csv(output_dir / 'container_counts.csv', index=False)
port_signals.to_csv(output_dir / 'port_signals.csv', index=False)
global_signals.to_csv(output_dir / 'global_signals.csv', index=False)

print(f"Results saved to {output_dir}")
print(f"\nSummary:")
print(f"- Total observations: {len(df_containers)}")
print(f"- Ports analyzed: {df_containers['port'].nunique()}")
print(f"- Date range: {df_containers['timestamp'].min()} to {df_containers['timestamp'].max()}")
print(f"- Average daily global volume: {global_signals['container_count'].mean():.0f} containers")